In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import fpgrowth, association_rules
from scipy import sparse

In [3]:

data1=pd.read_csv(r"C:\Users\shree\OneDrive\Documents\order_products__train.csv")

In [5]:
data1.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [6]:
small_train = data1[['order_id', 'product_id']]
small_train_split = (small_train[:461543], small_train[461543:461543*2-1], small_train[461543*2-1:])
pivots = []
for df in small_train_split:
    pvt = ~(df.pivot(index='order_id', columns='product_id', values='product_id').isna())
    pivots.append(pvt.astype(pd.SparseDtype(bool)))
del pvt

product_cols = sorted(small_train.product_id.unique())

In [7]:
for i in range(len(pivots)):
    
    pivots[i] = pivots[i].reindex(columns=product_cols, fill_value=False).astype(pd.SparseDtype(bool))
    pivots[i] = sparse.csr_matrix(pivots[i])
# concat vertically
pivots = sparse.vstack(pivots)

In [8]:
truth_table = pd.DataFrame(pivots.todense(), index=small_train.order_id.unique(), columns=product_cols)

In [9]:
frequent_itemsets = fpgrowth(truth_table, min_support=5/len(truth_table), use_colnames=True)

In [10]:
frequent_itemsets

,support,itemsets
0,0.117980,(13176)
1,0.055583,(47209)
2,0.018391,(49683)
3,0.015190,(22035)
4,0.008094,(10246)
...,...,...
861871,0.000053,"(8833, 9497)"
861872,0.000038,"(9497, 1134)"
861873,0.000038,"(9497, 8833, 1134)"
861874,0.000038,"(15317, 12902)"


In [11]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.8)

In [89]:
 print("μ number of consequents:", rules['consequents'].apply(len).mean())
rules

μ number of consequents: 1.0391897394136809


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(47626, 49683, 4605, 21903)",(24852),0.000076,0.142719,0.000069,0.900000,6.306104,0.000058,8.572811
1,"(26209, 49683, 28204, 16797)",(24852),0.000046,0.142719,0.000038,0.833333,5.838985,0.000032,5.143687
2,"(49683, 39275, 48679)",(24852),0.000046,0.142719,0.000038,0.833333,5.838985,0.000032,5.143687
3,"(27104, 49683, 24964, 47766)",(24852),0.000038,0.142719,0.000038,1.000000,7.006782,0.000033,inf
4,"(42265, 40706, 49683, 24852)",(21903),0.000046,0.074568,0.000038,0.833333,11.175474,0.000035,5.552592
...,...,...,...,...,...,...,...,...,...
29467,(26460),(38936),0.000046,0.000099,0.000038,0.833333,8410.833333,0.000038,5.999406
29468,(3458),(24852),0.000053,0.142719,0.000046,0.857143,6.005813,0.000038,6.000968
29469,"(9497, 1134)",(8833),0.000038,0.000114,0.000038,1.000000,8747.266667,0.000038,inf
29470,(15317),(12902),0.000038,0.000122,0.000038,1.000000,8200.562500,0.000038,inf


In [12]:
type(rules)

pandas.core.frame.DataFrame

In [14]:
data=pd.read_csv(r"C:\Users\shree\OneDrive\Documents\products.csv")
data.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [41]:
r=list(rules['antecedents'][1])
k=r[0]
data.query('product_id==47627')['product_name']

47626    Salsa Ranch Yogurt Dressing
Name: product_name, dtype: object

In [45]:
data.iloc[47620:47630,:]

,product_id,product_name,aisle_id,department_id
47620,47621,Chocolate Strong Dark,45,19
47621,47622,Wild Yam Soba Noodles,66,6
47622,47623,Seasoning Allspice,104,13
47623,47624,Crunchy Maple Brown Sugar Granola Bars,3,19
47624,47625,Fat Free French Vanilla Coffee Creamer,53,16
47625,47626,Large Lemon,24,4
47626,47627,Salsa Ranch Yogurt Dressing,89,13
47627,47628,Tart Cherry Juice Light,98,7
47628,47629,Annie's® Homegrown Gluten Free Double Chocolat...,3,19
47629,47630,Medium Navel Orange,24,4


In [46]:
data.loc[data['product_id']==r[0],'product_name'].iloc[0]

'Limes'